In [18]:
import json, os
from libs.utils.python import load_local_settings
load_local_settings()
import httpx
import adaptive_cards.card_types as types
from adaptive_cards.card import AdaptiveCard
from adaptive_cards.elements import TextBlock, Image
from adaptive_cards.containers import Container, ContainerTypes, ColumnSet, Column

In [19]:
ingress = {
    "function_name":"example-function",
    "instance_id":"test-123",
    "error":"Ice cream machine broke",
    "icon_url":"https://img.icons8.com/?size=77&id=16044&format=png",
    "webhook":os.environ["EXCEPTIONS_WEBHOOK_DEVOPS"]
}

In [20]:
# initialize the object that will contain all card elements
containers: list[ContainerTypes] = []

# build a header with title and icon
containers.append(
    Container(
        items=[
            ColumnSet(
                columns=[
                    Column(
                        items=[
                            TextBlock(
                                text="Azure Functions Error",
                                size=types.FontSize.LARGE,
                            )
                        ],
                        width="stretch",
                    ),
                    Column(
                        items=[Image(url=ingress['icon_url'], width="25px")],
                        rtl=True,
                        width="auto",
                    ),
                ]
            )
        ],
        style=types.ContainerStyle.EMPHASIS,
        bleed=True,
    )
)

# build a summary section with meta information
containers.append(
    Container(
        items=[
            TextBlock(
                text="Summary",
                size=types.FontSize.MEDIUM,
                weight='Bolder'
            ),
            ColumnSet(
                columns=[
                    Column(
                        items=[
                            TextBlock(text="Function Name"),
                            TextBlock(text="Instance ID"),
                        ],
                        width="100px"
                    ),
                    Column(
                        items=[
                            TextBlock(text=ingress['function_name']),
                            TextBlock(text=ingress['instance_id']),
                        ],
                        spacing=types.Spacing.MEDIUM,
                        rtl=False,
                    ),
                ],
                separator=True,
            )
        ],
        spacing=types.Spacing.SMALL,
    )
)

# build a details section with a truncated error trace
containers.append(
    Container(  
        items = [
            TextBlock(
                text="Details",
                size=types.FontSize.MEDIUM,
                weight='Bolder'
            ),
            TextBlock(
                text=ingress['error'],
                size=types.FontSize.SMALL,
                wrap=True,
                max_lines=10,
                separator=True
            ),
        ]
    )
)

# build Adaptive card from container elements
card = AdaptiveCard.new().version("1.5").add_items(containers).create()

In [ ]:
# build Adaptive card from container elements
print({"$"+k if k=='schema' else k:v for k,v in json.loads(card.to_json()).items()})

In [22]:
httpx.Client(timeout=None).request(
    method="POST",
    url=ingress['webhook'],
    json={
        "type": "message",
        "attachments": [
            {
                "contentType": "application/vnd.microsoft.card.adaptive",
                "contentUrl": None,
                "content": {"$"+k if k=='schema' else k:v for k,v in json.loads(card.to_json()).items()} # key exports as "schema", but it needs to be "$schema"
            }
        ],
    },
).status_code

200